In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
# Disable TensorFlow Caching
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

# Ensure limited GPU memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Batch sizes for testing
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 10  # Runs per batch size

# Load NASNetMobile model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow function to avoid extra optimizations
@tf.function(experimental_relax_shapes=True)
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 14:48:06.096660: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 14:48:06.096677: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 14:48:06.096681: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 14:48:06.096879: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 14:48:06.096890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Create random input tensor
        input_tensor = tf.random.uniform((batch_size, 224, 224, 3), dtype=tf.float32)
        
        with tf.device(device):
            #layer_times = {}
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

            # Manually clear TensorFlow cache after each batch run
            del input_tensor
            tf.keras.backend.clear_session

    return results

In [4]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []
for device in devices:
    for batch_size in BATCH_SIZES:
        try:
            tf.profiler.experimental.start(logdir = f"logs/Noise/NoCache/{device[1:4]}/{batch_size}", 
                                           options=tf.profiler.experimental.ProfilerOptions(host_tracer_level = 3,
                                                                                            device_tracer_level = 3))
            final_results.extend(benchmark_model(device, batch_size))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 14:48:08.390120: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:48:08.390130: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 14:48:10.053713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 14:48:11.481018: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:48:12.064447: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:48:12.065412: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/NoCache/CPU/1/plugins/profile/2025_02_05_14_48_12/C17586.xplane.pb
2025-02-05 14:48:12.128957: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:48:12.128977: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 4


2025-02-05 14:48:15.827100: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:48:16.424418: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:48:16.424706: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/NoCache/CPU/4/plugins/profile/2025_02_05_14_48_16/C17586.xplane.pb
2025-02-05 14:48:16.488587: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:48:16.488601: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


...

2025-02-05 14:49:47.337120: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:49:47.615924: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:49:47.616214: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/NoCache/GPU/16/plugins/profile/2025_02_05_14_49_47/C17586.xplane.pb
2025-02-05 14:49:47.648761: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:49:47.648770: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 32


2025-02-05 14:50:06.340207: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:50:06.738265: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:50:06.738627: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/NoCache/GPU/32/plugins/profile/2025_02_05_14_50_06/C17586.xplane.pb
2025-02-05 14:50:06.785584: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:50:06.785596: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 64


2025-02-05 14:50:49.341896: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:50:50.523576: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:50:50.526956: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/NoCache/GPU/64/plugins/profile/2025_02_05_14_50_50/C17586.xplane.pb


In [5]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.043103
1,/CPU:0,1,0.042738
2,/CPU:0,1,0.041926
3,/CPU:0,1,0.042825
4,/CPU:0,1,0.042328


In [6]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                              
                         min      mean       var       max
device batch_size                                         
/CPU:0 1            0.040030  0.041796  0.000001  0.043103
       4            0.065577  0.068976  0.000010  0.075864
       8            0.092095  0.099208  0.000027  0.108893
       16           0.154609  0.162060  0.000040  0.173840
       32           0.299390  0.333876  0.000781  0.385820
       64           1.066665  1.481535  0.035918  1.759905
/GPU:0 1            0.086622  0.092377  0.000078  0.116993
       4            0.090770  0.096814  0.000046  0.114641
       8            0.092988  0.095316  0.000017  0.106613
       16           0.095646  0.099498  0.000045  0.114733
       32           0.144419  0.237735  0.025496  0.597451
       64           0.994831  1.233242  0.020072  1.452809